In [1]:
!pip install selenium
!pip install webdriver_manager
!pip install docker
!pip install requests

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


##  Get the container ID

In [2]:
import subprocess

# Get the first container ID
#get_cont_id_cmd = "docker ps -q | head -n 1"
get_cont_id_cmd = "/usr/local/bin/docker ps -q | head -n 1"
cont_id = subprocess.check_output(get_cont_id_cmd, shell=True).decode('utf-8').strip()

print("Container ID:", cont_id)

Container ID: 


Cannot connect to the Docker daemon at unix:///Users/zhangyiqin/.docker/run/docker.sock. Is the docker daemon running?


## Get the Jupyter lab token

In [3]:
get_token_cmd = f"docker exec {cont_id} jupyter lab list 2>&1 | grep -o 'token=[^ ]*' | cut -d= -f2"
token = subprocess.check_output(get_token_cmd, shell=True).decode('utf-8').strip()

print("Jupyter Lab Token:", token)

Jupyter Lab Token: 


##  Run the rise_test.py 

In [4]:
# Run the rise_test.py with the token
if token:
    run_script_cmd = f"python3 rise_test.py {token}"
    script_output = subprocess.check_output(run_script_cmd, shell=True).decode('utf-8')
    print("Script Output:\n", script_output)    
else:
    print("Token not found. Check if Jupyter Lab is running in the container.")

Token not found. Check if Jupyter Lab is running in the container.


## Test the RISE

In [5]:
import requests
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import platform


In [6]:
def setup_driver():
    chrome_options = Options()
    if platform.system() in ['Windows', 'Darwin']:
        chrome_options.add_experimental_option('detach', True)
    elif platform.system() == 'Linux':
        chrome_options.add_argument('--headless')
    else:
        raise Exception("Unknown OS")
    return webdriver.Chrome(service=ChromiumService(ChromeDriverManager().install()), options=chrome_options)

In [7]:
base_url = "http://localhost:8888"  # Base URL of your JupyterLab server
image_path = "/lab/images/sample-image.jpg"  # Extracted relative path
full_image_url = base_url + image_path
image_url="https://raw.githubusercontent.com/YiqinZhang/ope-project/e9ea65fa2317b3c39d70984875ce2885dde8e599/content/images/sample-image.jpg"

In [8]:
def create_jupyter_notebook_from_github(github_url, local_file_name):
    try:
        response = requests.get(github_url)
        response.raise_for_status()  # Raise an error for bad status codes
        notebook_content = response.json()
        with open(local_file_name, 'w', encoding='utf-8') as notebook_file:
            json.dump(notebook_content, notebook_file)
        print(f"Notebook created: {local_file_name}")
    except requests.RequestException as e:
        print(f"Failed to download the notebook: {e}")

In [9]:
import os
def download_image(image_url, local_img_path):
    response = requests.get(image_url, stream=True)

    if response.status_code == 200:
        os.makedirs(os.path.dirname(local_img_path), exist_ok=True)
        with open(local_img_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)
        print(f"Image downloaded successfully: {local_img_path}")
    else:
        print(f"Failed to download the image: {image_url}")

image_url = 'https://raw.githubusercontent.com/YiqinZhang/ope-project/e9ea65fa2317b3c39d70984875ce2885dde8e599/content/images/sample-image.jpg'
download_image(image_url, 'images/sample-image.jpg')


Image downloaded successfully: local_images/sample-image.jpg


In [10]:
notebook_github_url = "https://raw.githubusercontent.com/OPEFFORT/ope-project/main/content/test_book/02_slides_template/layout_example.ipynb"
local_notebook_name = "downloaded_notebook.ipynb"
create_jupyter_notebook_from_github(notebook_github_url, local_notebook_name)

Notebook created: downloaded_notebook.ipynb


In [11]:
def open_file_in_jupyterlab(url, file_name, driver):
    try:
        driver.get(url)
        # Wait for the file browser to be available
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".jp-DirListing-content"))
        )

        # Find the file in the file browser
        file_element = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, f"//span[contains(text(), '{file_name}') and @class='jp-DirListing-itemText']"))
        )

        # Double click on the file to open it
        driver.execute_script("""
            var evt = new MouseEvent('dblclick', {
                bubbles: true,
                cancelable: true,
                view: window
            });
            arguments[0].dispatchEvent(evt);
        """, file_element)

        print(f"File '{file_name}' opened successfully.")
    except TimeoutException:
        print(f"Failed to open file '{file_name}'. File not found or JupyterLab did not load properly.")

In [12]:
def take_screenshot(driver, filename):
    try:
        driver.save_screenshot(filename)
        print(f"Saved screenshot: {filename}")
    except Exception as e:
        print(f"Error taking screenshot: {e}")


In [13]:
def find_element_and_click(driver, xpath):
    try:
        element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
        # element = driver.find_element(By.XPATH, xpath)
        element.click()
        return element
    except Exception as e:
        print(f"Error finding/clicking element: {e}")


## Test1: Refactor the Previous Test for Suitability with the Launcher

In [14]:
def test_rise_extension(token):
    driver = setup_driver()
    url = f"http://127.0.0.1:8888/lab?token={token}"

    try:
        driver.get(url)
        driver.implicitly_wait(45)
        # Main actions and interactions
        take_screenshot(driver, 'main_screenshot.png')

        find_element_and_click(driver, '//div[@data-category="Notebook"]')
        take_screenshot(driver, 'new_nb_screenshot.png')

        find_element_and_click(driver, '//button[@data-command="RISE:preview"]')
        take_screenshot(driver, 'rise_screenshot.png')

        toolbar = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[@class='lm-Widget jp-Toolbar' and @role='navigation']")))
        if toolbar:
            fullscreen_button = toolbar.find_element(By.XPATH, ".//button[@title='Open the slideshow in full screen']")
            fullscreen_button.click()
            is_fullscreen = driver.execute_script("return document.fullscreenElement !== null")
            print("Fullscreen button is functioning: ", is_fullscreen)
        else:
            print("Toolbar not found")

        print("RISE Extension is working!")
    except Exception as e:
        print(f"An error occurred during test: {e}")
    finally:
        # Clean up
        driver.switch_to.window(driver.window_handles[0])
        driver.refresh()
        driver.quit()

In [15]:
# replace token with your token
token = "cc2f522441ace86a2bbe4fa85e17f3a488fb181d1538da19"
test_rise_extension(token)

Saved screenshot: main_screenshot.png
Error finding/clicking element: Message: 
Stacktrace:
0   chromedriver                        0x0000000102673d28 chromedriver + 4795688
1   chromedriver                        0x000000010266b2b3 chromedriver + 4760243
2   chromedriver                        0x000000010224488d chromedriver + 407693
3   chromedriver                        0x00000001022933d9 chromedriver + 730073
4   chromedriver                        0x0000000102293591 chromedriver + 730513
5   chromedriver                        0x00000001022d97e4 chromedriver + 1017828
6   chromedriver                        0x00000001022baccd chromedriver + 892109
7   chromedriver                        0x00000001022d6b2b chromedriver + 1006379
8   chromedriver                        0x00000001022baa73 chromedriver + 891507
9   chromedriver                        0x0000000102285143 chromedriver + 672067
10  chromedriver                        0x000000010228631e chromedriver + 676638
11  chromedri

## Test 2: Downloaded Testbook Slides for the RISE Test

In [16]:
def test_rise(token, notebook_github_url, local_notebook_name):
    driver = setup_driver()
    create_jupyter_notebook_from_github(notebook_github_url, local_notebook_name)
    open_file_in_jupyterlab(f"http://127.0.0.1:8888/lab?token={token}", local_notebook_name,driver)

    try:
        WebDriverWait(driver, 45)
        take_screenshot(driver, 'main_screenshot.png')

        # Correct the XPath and check if the "Launcher" tab is present
        try:
            # launcher_element = driver.find_element(By.XPATH, '//div[@data-category="Notebook"]')
            launcher_element = driver.find_element(By.XPATH, "//div[contains(@class, 'lm-TabBar-tabLabel') and contains(@class, 'p-TabBar-tabLabel') and text()='Launcher']")
            is_launcher_page = True
        except NoSuchElementException:
            is_launcher_page = False 
        
        if is_launcher_page:
            print("On the Launcher page")
            find_element_and_click(driver, '//div[@data-category="Notebook"]')
        else:
            print("Not on the Launcher page, trying to open the notebook directly")
            open_file_in_jupyterlab(f"http://127.0.0.1:8888/lab?token={token}", local_notebook_name,driver)
     
        take_screenshot(driver, 'new_nb_screenshot.png')
        
        find_element_and_click(driver, '//button[@data-command="RISE:preview"]')        
        take_screenshot(driver, 'rise_screenshot.png')

        fullscreen_button = find_element_and_click(driver, ".//button[@title='Open the slideshow in full screen']")
        is_fullscreen = driver.execute_script("return document.fullscreenElement !== null")
        print("Fullscreen button is functioning")

    finally:
        # Clean up  
        driver.switch_to.window(driver.window_handles[0])
        driver.refresh()
        WebDriverWait(driver, 30)
        driver.quit()

In [17]:
# replace token with your token
token = "cc2f522441ace86a2bbe4fa85e17f3a488fb181d1538da19"
test_rise(token, notebook_github_url, local_notebook_name)

Notebook created: downloaded_notebook.ipynb
Failed to open file 'downloaded_notebook.ipynb'. File not found or JupyterLab did not load properly.
Saved screenshot: main_screenshot.png
Not on the Launcher page, trying to open the notebook directly


UnexpectedAlertPresentException: Alert Text: {Alert text : 
Message: unexpected alert open: {Alert text : }
  (Session info: chrome=119.0.6045.159)
Stacktrace:
0   chromedriver                        0x00000001098d6d28 chromedriver + 4795688
1   chromedriver                        0x00000001098ce2b3 chromedriver + 4760243
2   chromedriver                        0x00000001094a788d chromedriver + 407693
3   chromedriver                        0x000000010953a019 chromedriver + 1007641
4   chromedriver                        0x000000010951da73 chromedriver + 891507
5   chromedriver                        0x00000001094e8143 chromedriver + 672067
6   chromedriver                        0x00000001094e931e chromedriver + 676638
7   chromedriver                        0x0000000109897795 chromedriver + 4536213
8   chromedriver                        0x000000010989c853 chromedriver + 4556883
9   chromedriver                        0x000000010987d001 chromedriver + 4427777
10  chromedriver        

## Test 3: Create slides and test RISE

In [18]:
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell

# Define the cells with their content and metadata
cells = [
    new_markdown_cell("# Example Slidedeck", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("<hr>", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("**0) Blank**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("**BODY**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("We will try to illustrate single-column slides like this one.", metadata={"slideshow": {"slide_type": "slide"}}),
    new_markdown_cell("<hr>", metadata={"slideshow": {"slide_type": "skip"}}),  
    new_markdown_cell(source="**1) Image Only**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell(source="**BODY**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell(source='<img src="../../images/sample-image.jpg" width = "100%">', 
                      metadata={"slideshow": {"slide_type": "slide"}}),
    new_markdown_cell(source="**NOTE**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell(source="- Using % of slide to scale the image", 
                      metadata={"slideshow": {"slide_type": "notes"}}),
    new_markdown_cell(source="- - - ", metadata={"slideshow": {"slide_type": "skip"}}),  
    new_markdown_cell("**7)Title Slide**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("# Title", metadata={"slideshow": {"slide_type": "slide"}}),
    new_markdown_cell("**NOTE**", metadata={"slideshow": {"slide_type": "skip"}}),
    new_markdown_cell("- a Note that will display in the notes view", metadata={"slideshow": {"slide_type": "notes"}}),
    new_markdown_cell("This is the end of the document.", metadata={"slideshow": {"slide_type": "skip"}, "tags": ["remove-cell"]}),
]

In [19]:
def test_rise_slide(token):
    driver = setup_driver()
    url = f"http://127.0.0.1:8888/lab?token={token}"

    try:
        driver.get(url)
        WebDriverWait(driver, 45)
        take_screenshot(driver, 'main_screenshot.png')

        # Check if on the Launcher page
        try:
            driver.find_element(By.XPATH, "//div[contains(@class, 'lm-TabBar-tabLabel') and contains(@class, 'p-TabBar-tabLabel') and text()='Launcher']")
            print("On the Launcher page")
            find_element_and_click(driver, '//div[@data-category="Notebook"]')
        except NoSuchElementException:
            print("Not on the Launcher page")

        # Create a new notebook object
        nb = new_notebook()
        # Add cells to the notebook
        nb['cells'] = cells

        # Write the notebook to a file
        with open('example_notebook.ipynb', 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)

        print("Notebook created successfully.")

        take_screenshot(driver, 'new_nb_screenshot.png')

        find_element_and_click(driver, '//button[@data-command="RISE:preview"]')
        take_screenshot(driver, 'rise_screenshot.png')

        toolbar = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[@class='lm-Widget jp-Toolbar' and @role='navigation']")))
        if toolbar:
            fullscreen_button = toolbar.find_element(By.XPATH, ".//button[@title='Open the slideshow in full screen']")
            fullscreen_button.click()
            is_fullscreen = driver.execute_script("return document.fullscreenElement !== null")
            print("Fullscreen button is functioning: ", is_fullscreen)
        else:
            print("Toolbar not found")

        print("RISE Extension is working!")

    except Exception as e:
        print(f"An error occurred during test: {e}")
    finally:
        # Clean up
        driver.switch_to.window(driver.window_handles[0])
        driver.refresh()
        driver.quit()

In [20]:
# replace token with your token
token = "cc2f522441ace86a2bbe4fa85e17f3a488fb181d1538da19"
test_rise_slide(token)

Saved screenshot: main_screenshot.png
Not on the Launcher page
Notebook created successfully.
Saved screenshot: new_nb_screenshot.png
Saved screenshot: rise_screenshot.png
Fullscreen button is functioning:  False
RISE Extension is working!
